## 9.3.8 배열
## 실습문제 : 배열 사용

In [1]:
%%writefile src/ArrayTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
// pragma solidity ^0.8.0;
pragma solidity ^0.6.4;

contract ArrayTest {
    uint[3] ages = [15, 25, 35];
    int[] marks; 

    function updateAges(uint index, uint val) public {
        if(index >= 0 && index <= 2) {
            ages[index] = val;
        }
    }

    function initMarks() public {
        marks = new int[](5); // default 0
    }

    function appendMark(int mark) public { 
        marks.push(mark);
    }

    function popMark() public {
        marks.pop();
    }

    function getMarks() public view returns(int[] memory) {
        return marks;
    }

    function getAges() public view returns(uint[3] memory) {
        return ages;
    }

    function getLenOfArr() pure public returns(uint) {
        uint8[3] memory intArr = [0, 1, 2];
        return intArr.length;
    }
}

Overwriting src/ArrayTest.sol


In [2]:
!solc.exe src/ArrayTest.sol

Compiler run successful, no output requested.


## 9.3.9 계정 주소
## 실습문제 : 송금

In [3]:
%%writefile src/AddressTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
// pragma solidity ^0.8.0;
pragma solidity ^0.6.4;

contract ArrayTest {
    address owner;
    address payable receiver;
    uint balanceOfOwner;
    
    constructor() public {
        owner = msg.sender;
        balanceOfOwner = owner.balance;
    }

    function deposit() payable public {

    }

    function setReceiver(address payable addr) public {
        receiver = addr;
    }

    function getReceiver() view public returns(address) {
        return receiver;
    }

    function getBalanceOfThis() public view returns(uint) {
        return address(this).balance;
    }

    function getBalanceOfOwner() public view returns(uint) {
        return owner.balance;
    }

    function send() public payable {
        require(receiver.send(111));
    }

    function transfer() public payable {
        receiver.transfer(11111);
    }

    function callValue() public payable {
        (bool success, ) = receiver.call{value: 11111}("");
        require(success, "transfer call failed.");
        (success, ) = receiver.call{gas: 10, value: 11111}("");
        require(success, "transfer call failed.");
    }
}

Writing src/AddressTest.sol


In [4]:
!solc.exe src/AddressTest.sol

Compiler run successful, no output requested.


## 9.6 연산자
## 실습문제 : 주소 계정에서 입출금

In [11]:
%%writefile src/MyBank.sol
//SPDX-License-Identifier: GPL-3.0-or-later
// pragma solidity ^0.8.0;
pragma solidity ^0.6.4;

contract MyBank {
    address owner;
    uint balance;

    constructor() public {
        owner = msg.sender;
        balance = address(this).balance;
    }

    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }

    function withdraw(uint amount) public payable {
        balance -= amount;
        payable(owner).transfer(amount);
    }

    function transferTo(address payable receiver, uint amount) public payable {
        balance -= amount;
        receiver.transfer(amount);
    }

    function getBalance() public view returns(uint) {
        return balance;
    }

    function getBalanceOfThis() public view returns(uint) {
        return address(this).balance;
    }

    function getBalanceOfOwne() public view returns(uint) {
        return owner.balance;
    }
}

Overwriting src/MyBank.sol


In [12]:
!solc.exe --optimize --combined-json abi,bin src/MyBank.sol > src/MyBank.json

In [17]:
%%writefile src/MyBankDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./MyBank.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas: 364124}, function(err, transactionHash) {
            if(!err) {
                console.log("hash: " + transactionHash);
            }
        });
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Overwriting src/MyBankDeploy.js


In [18]:
!node src/MyBankDeploy.js

- contract name :  [ 'src/MyBank.sol:MyBank' ]
Deploying the contract from 0xaD613993B7400B5494F5978723599913165420e7
hash: 0x389313fc7a487bd1384fc24bd97773dec930bc07ddb3afd328e6c2a78c79fac0
---> The contract deployed to : 0xA26676c2906FFF4C84A9AFED6f4af7a1042C5fd3


In [19]:
%%writefile src/MyBankUse.js
var Web3 = require('web3');
var _abiBinJson = require('./MyBank.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

var myBank = new web3.eth.Contract(_abiArray, "0xA26676c2906FFF4C84A9AFED6f4af7a1042C5fd3");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Call from : " + accounts[0]);
    myBank.methods.getBalance().call().then(console.log);
    myBank.methods.deposit(1111).send({from: accounts[0], gas: 80000, value: 1111});
    myBank.methods.getBalance().call().then(console.log);
}

doIt();

Writing src/MyBankUse.js


In [21]:
!node src/MyBankUse.js

Call from : 0xaD613993B7400B5494F5978723599913165420e7
1111
1111


## 9.7 형변환
## 실습문제 : 형변환

In [22]:
%%writefile src/DataConversionTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract DataConversionTest {
    function convertInt8ToInt() pure public returns(int) {
        int8 i8 = 20;
        return int(i8);
    }

    function convertIntToBytes32() pure public returns(bytes32) {
        uint x = 20; 
        //0x0000000000000000000000000000000000000000000000000000000000000014
        return bytes32(x);
    }

    function convertInt64ToBytes8() pure public returns(bytes8) {
        uint64 x = 10;
        return bytes8(x);
    }

    function convertIntToBytes() pure public returns(bytes memory) {
        uint x = 20;
        bytes32 y = bytes32(x);
        bytes memory z = new bytes(32);
        for (uint i = 0; i < 32; i++) {
            z[i] = y[i];
        }
        return z;
    }

    function convertStringToBytes() pure public returns(bytes memory) {
        string memory s = "hello";
        return bytes(s);
    }

    function convertBytes4ToBytes2() pure public returns(bytes2) {
        bytes4 b4 = 0x68656c6c;
        return bytes2(b4);
    }

    function convertBytes4ToInt32() pure public returns(uint32) {
        bytes4 b4 = 0x68656c6c;
        return uint32(b4);
    }
}

Writing src/DataConversionTest.sol


In [23]:
!solc.exe src/DataConversionTest.sol

Compiler run successful, no output requested.


## 9.8 전역 변수
## 실습문제 : 전역변수

In [24]:
%%writefile src/GlobalVarsTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4; 

contract GlobalVarsTest {
    function getEther() pure public returns(uint) {
        return 1 ether;
    }

    function getDays() view public returns(uint, uint) {
        return(block.timestamp, 1 days);
    }

    function getMsgValue() view internal returns(uint) {
        return msg.value;
    }

    function getMsgSender() view public returns(address) {
        return msg.sender;
    }

    function getCoinbase() view public returns(address) {
        return block.coinbase;
    }

    function getBlockNumber() view public returns(uint) {
        return block.number;
    }

    function getBlockTimeStamp() view public returns(uint) {
        return block.timestamp;
    }
}

Writing src/GlobalVarsTest.sol


In [25]:
!solc.exe src/GlobalVarsTest.sol

Compiler run successful, no output requested.


## 5.9 Opcode 읽어보기
## 실습문제 : 송금 더 해보기

In [26]:
%%writefile src/BankV2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4; 

contract BankV2 {
    address owner;
    uint balance;
    
    constructor() public {
        owner = msg.sender;
        balance = 0;
    }

    event Sent(address from, address to, uint amount);

    function forwardTo(address payable _receiver) public payable {
        require(msg.sender == owner);
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }

    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }

    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }

    function withdrawAll() public {
        balance -= address(this).balance;
        require(msg.sender == owner);
        msg.sender.transfer(address(this).balance);
    }
}

Writing src/BankV2.sol


In [27]:
!solc --optimize --combined-json abi,bin src/BankV2.sol > src/BankV2.json

In [30]:
%%writefile src/BankV2Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abiBinJson = require('./BankV2.json');
contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas: 1000000, gasPrice: '1000000000'}, function(err, transactionHash) {
            if(!err) {
                console.log("hash : " + transactionHash);
            }
        });
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Overwriting src/BankV2Deploy.js


In [31]:
!node src/BankV2Deploy.js

- contract name :  [ 'src/BankV2.sol:BankV2' ]
Deploying the contract from 0xaD613993B7400B5494F5978723599913165420e7
hash : 0x6cda99afd58e386a2c720fb23add831d07d16de467e8bada8da8dfc0f653e25c
---> The contract deployed to : 0x54f27773f007028037A2f47C1800FBc078DeF63a


In [38]:
%%writefile src/BankV2Use.js
var Web3 = require('web3');
var _abiBinJson = require('./MyBank.json');
//var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider('ws://localhost:8345'));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

var bank = new web3.eth.Contract(_abiArray, "0x54f27773f007028037A2f47C1800FBc078DeF63a");
var event = bank.events.Sent({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account 0 : " + accounts[0]);
    const balance0Before = await web3.eth.getBalance(accounts[0]);
    console.log("Balance0 before : " + balanc0Before);
    const balance1Before = await web3.eth.getBalance(accounts[1]);
    console.log("Balacne1 before : " + balance1Before);
    
    bank.methods.getBalance().call().then(function(bal) {
        console.log("Contract bal before deposit : " + bal[0] + " this.bal : " + bal[1]);
    });
    
    await bank.methods.deposit(1111).send({from: accounts[0], gas: 80000, value: 1111});
    bank.methods.getBalance().call().then(function(bal) {
        console.log("Contract bal after deposit : " + bal[0] + " this.bal : " + bal[1]);
    })
    
    const forward = await bank.methods.forwardTo(accounts[1])
        .send({from: accounts[0], gas: 100000, value: 555});
    
    console.log("---> forwardTo called : " + JSON.stringify(forward.events.Sent.returnValues));
    bank.methods.getBalance().call().then(function(bal) {
        console.log("Contract balance after forwardTo : " + bal[0] + " this.bal : " + bal[1]);
    });
    
    const balance0After = await web3.eth.getBalance(accounts[0]);
    console.log("Balance0 after: " + balance0After);
    console.log("Balance0 diff: " + (balance0After - balance0Before));
    const balance1After = await web3.eth.getBalance(accounts[1]);
    console.log("Balance1 after: " + balance1After);
    console.log("Balance1 diff: " + (balance1After - balance1Before));

    const withdraw = await bank.methods.widthdrawAll().send({from: accounts[0],gas:100000});
    bank.methods.getBalance().call().then(function(bal) {
        console.log("Contract balance after withdrawAll: " + bal[0] + " this.bal:" + bal[1]);
    });
    const balance0AfterWithdrawAll = await web3.eth.getBalance(accounts[0]);
    console.log("Balance0 after withdrawAll: " + balance0AfterWithdrawAll);
    console.log("Balance0 diff after withdrawAll: " + (balance0After - balance0AfterWithdrawAll));
}

doIt();

Overwriting src/BankV2Use.js


In [39]:
!node src/BankV2Use.js

- contract name :  [ 'src/MyBank.sol:MyBank' ]


C:\Users\youve\Code\201810820\src\BankV2Use.js:12
var event = bank.events.Sent({fromBlock: 0}, function (error, result) {
                        ^

TypeError: bank.events.Sent is not a function
    at Object.<anonymous> (C:\Users\youve\Code\201810820\src\BankV2Use.js:12:25)
    at Module._compile (node:internal/modules/cjs/loader:1103:14)
    at Object.Module._extensions..js (node:internal/modules/cjs/loader:1157:10)
    at Module.load (node:internal/modules/cjs/loader:981:32)
    at Function.Module._load (node:internal/modules/cjs/loader:822:12)
    at Function.executeUserEntryPoint [as runMain] (node:internal/modules/run_main:77:12)
    at node:internal/main/run_main_module:17:47


## 10.4 함수의 제약조건을 지정하는 modifier
## 실습문제 : 은행 BankV3

In [1]:
%%writefile src/BankV3.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract BankV3 {
    address owner;
    uint balance;
    uint256 timeToDeposit;

    event PrintLog(string);
    event Sent(address from, address to, uint amount);

    constructor() public {
        owner = msg.sender;
        balance = 0;
    }

    fallback() external {
        emit PrintLog("Fallback called");
    }

    function forwardTo(address payable _receiver) public payable onlyOwner {
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }

    function getBalance() public view returns(uint, uint) {
        return(balance, address(this).balance);
    }

    function deposit(uint amount) public payable onlyAfter {
        timeToDeposit = block.timestamp + 10 seconds;
        require(msg.value == amount);
        balance += amount;
    }

    function witdrawAll() public onlyOwner minBalance {
        balance -= address(this).balance;
        payable(msg.sender).transfer(address(this).balance);
    }

    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }

    modifier onlyAfter {
        require(block.timestamp >= timeToDeposit);
        _;
    }

    modifier minBalance {
        require(address(this).balance > 101 wei);
        _;
    }
}

Writing src/BankV3.sol


In [2]:
!solc.exe --optimize --combined-json abi,bin src/BankV3.sol > src/BankV3.json

In [3]:
%%writefile src/BankV3Deploy.js
var Web3 = require('web3');
var _abiBinJson = require('./BankV3.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from : " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
            if(!err) {
                console.log("hash " + transactionHash);
            }
        })
    console.log("---> The contract deployed to " + deployed.options.address);
}

deploy();

Writing src/BankV3Deploy.js


In [5]:
!node src/BankV3Deploy.js

- contract name :  [ 'src/BankV3.sol:BankV3' ]
Deploying the contract from : 0x0B5d062d359699F97c93eB4C7B20D10C28D4ec21
hash 0x8e2b0009eced98c2c8a5746820b8cfa285100d456e3fe213f483dc4a434da733
---> The contract deployed to 0xe5bB9550c4680411999927C710f3947486d76600


In [8]:
%%writefile src/BankV3Use.js
var Web3 = require('web3');
var _abiBinJson = require('./BankV3.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

var bank = new web3.eth.Contract(_abiArray, "0xe5bB9550c4680411999927C710f3947486d76600");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account : " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before : " + balanceBefore);
    bank.methods.getBalance().call().then(console.log);
    await bank.methods.deposit(111).send({from: accounts[0], value: 111});
    bank.methods.getBalance().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));    
}

doIt();

Overwriting src/BankV3Use.js


In [9]:
!node src/BankV3Use.js

- contract name :  [ 'src/BankV3.sol:BankV3' ]
Account : 0x0B5d062d359699F97c93eB4C7B20D10C28D4ec21
Balance before : 999999433813999999889
Result { '0': '111', '1': '111' }
Balance after: 999999370841999999778
Balance diff: 62971999944704
Result { '0': '222', '1': '222' }


## 10.5.2 웹소켓
## 실습문제 : 간단한 이벤트 발생

In [10]:
%%writefile src/EventTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract EventTest {
    event MyLog(string my);
    function myFunction() public {
        emit MyLog("Hello World!");
    }
}

Overwriting src/EventTest.sol


In [11]:
!solc.exe --optimize --combined-json abi,bin src/EventTest.sol > src/EventTest.json

In [12]:
%%writefile src/EventTestDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./EventTest.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
            if(!err) {
                console.log("hash : " + transactionHash);
            }
        })
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Writing src/EventTestDeploy.js


In [13]:
!node src/EventTestDeploy.js

- contract name :  [ 'src/EventTest.sol:EventTest' ]
Deploying the contract from 0x0B5d062d359699F97c93eB4C7B20D10C28D4ec21
hash : 0x046266f4af166371e835d4c5e69f40d8f4f43ed78fcc112b1b57b1c6dbe15f9a
---> The contract deployed to : 0x13CA11BDf82ea804891b715FE0563DaF88Ab0D9E


In [17]:
%%writefile src/EventTestUse.js
var Web3 = require('web3');
var _abiBinJson = require('./EventTest.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

var _test = new web3.eth.Contract(_abiArray,"0x13CA11BDf82ea804891b715FE0563DaF88Ab0D9E");
var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt();

Overwriting src/EventTestUse.js


In [18]:
!node src/EventTestUse.js

- contract name :  [ 'src/EventTest.sol:EventTest' ]
Account: 0x0B5d062d359699F97c93eB4C7B20D10C28D4ec21
Balance before: 999999182755999999778
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 999999159952999999778
Balance diff: 22803000066048


## 실습문제 : 주문하면서 복수의 이벤트 사용

In [19]:
%%writefile src/OrderEvent.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract OrderEvent {
    uint unitPrice = 10;

    event OrderLog(string);
    event OrderLog(bytes2 _itemId, uint _value);
    event OrderLog(uint256 timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed _value);

    function order(bytes2 _itemId, uint quantity) public payable {
        uint256 orderTime = now;
        uint256 orderAmount = quantity * unitPrice;
        require(msg.value == orderAmount);
        emit OrderLog("ordered");
        emit OrderLog(orderTime);
        emit OrderLog(msg.sender, _itemId, msg.value);
    }
}

Writing src/OrderEvent.sol


In [20]:
!solc.exe --optimize --combined-json abi,bin src/OrderEvent.sol > src/OrderEvent.json

In [21]:
%%writefile src/OrderEventDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./OrderEvent.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        });
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/OrderEventDeploy.js


In [22]:
!node src/OrderEventDeploy.js

- contract name :  [ 'src/OrderEvent.sol:OrderEvent' ]
Deploying the contract from 0x0B5d062d359699F97c93eB4C7B20D10C28D4ec21
hash: 0x09149a0a90953c5a287a820aa1b7bbe1a0cc85f674fc4c8073c1a00d50693e7d
---> The contract deployed to: 0x10aDBeA225Fa793C350a554De25A75D6F287361A


In [23]:
%%writefile src/OrderEventUse.js
var Web3 = require('web3');
var _abiBinJson = require('./OrderEvent.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, "0x10aDBeA225Fa793C350a554De25A75D6F287361A");
    const accounts = await web3.eth.getAccounts();
    console.log("Account : " + accounts[0]);
    var event = _order.events.OrderLog({
        filter: {_from: accounts[0], _value: 30},
        fromBlock: 'latest', toBlock: 'pending'
    }, function(error, result) {
        if(!error) {
            console.log("Event fired : " + JSON.stringify(result.returnValues));
        }
    });
    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance Before : " + balanceBefore);
    my = await _order.methods.order("0x1234", 3)
        .send({from: accounts[0], gas: 100000, value:30});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    my = await _order.methods.order("0x1234", 4).send({from: accounts[0], gas: 100000, value:40});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    my = await _order.methods.order("0x1234", 10).send({from: accounts[0], gas: 100000, value:100});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));    
    process.exit(1); 
}

doIt();

Writing src/OrderEventUse.js


In [24]:
!node src/OrderEventUse.js

- contract name :  [ 'src/OrderEvent.sol:OrderEvent' ]
Account : 0x0B5d062d359699F97c93eB4C7B20D10C28D4ec21
Balance Before : 999998857202999999778
---> MyFunction called undefined
---> MyFunction called undefined
---> MyFunction called undefined
Balance after: 999998688098999999608
Balance diff: 169104000024576


## 실습문제 : Multiply7 연산 결과를 확인하려면 이벤트를 통해서 한다

In [1]:
%%writefile src/Multiply7Event.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Multiply7Event {
    event Print(address _addr, uint256 timestamp, uint res);
    //function multiply(uint input) public returns(uint) {
    function multiply(uint input) public {
        uint res = input * 7;
        emit Print(msg.sender, block.timestamp, res); //emit Print(msg.sender, now, res);
        //return res;
    }
}

Overwriting src/Multiply7Event.sol


In [2]:
!solc-windows.exe --optimize --combined-json abi,bin src/Multiply7Event.sol > src/Multiply7Event.json

In [7]:
%%writefile src/Multiply7EventDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./Multiply7Event.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/Multiply7EventDeploy.sol:Multiply7EventDeploy']
console.log("- contract name: ", contractName);
_abiArray=_abiBinJson.contracts[contractName].abi;
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin=_abiBinJson.contracts[contractName].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/Multiply7EventDeploy.js


In [8]:
!node src/Multiply7EventDeploy.js

- contract name:  [ 'src/Multiply7Event.sol:Multiply7Event' ]
Deploying the contract from 0x51967008fA944866Bb6Df61d1941E11760f33326
hash: 0x8d9aa92ae1b8c8459b7a391924364fbb139df40bc5b4e8deb93ac0f90c85079e
---> The contract deployed to: 0xF461d1Ae7c862bbDcc1a32AB904084F4a0B1CcE5


In [9]:
%%writefile src/Multiply7EventUse.js
var Web3=require('web3');
var _abiBinJson = require('./Multiply7Event.json');      //importing a javascript file

//var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/Multiply7Event.sol:Multiply7Event']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
var m7 = new web3.eth.Contract(_abiArray, '0xF461d1Ae7c862bbDcc1a32AB904084F4a0B1CcE5');
var event = m7.events.Print({fromBlock: 0}, function (error, result) {
    if (!error) {
        //console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
        console.log("Event fired: " + JSON.stringify(result.returnValues));
        //process.exit(1);
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    //const value = m7.methods.multiply(8).call();
    const value = m7.methods.multiply(8)
        .send({from: accounts[0]})
        //.then(function(value) {console.log("---> MyFunction called " + JSON.stringify(value) +
        //                               '\n---> '+ JSON.stringify(value.events.Print.returnValues));});
    console.log("---> MyFunction called " + JSON.stringify(value));
        //+ '\n---> '+ JSON.stringify(value.events.Print.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); //force exit to end waiting
}

doIt()

Overwriting src/Multiply7EventUse.js


In [12]:
!node src/Multiply7EventUse.js

Event fired: {"0":"0x51967008fA944866Bb6Df61d1941E11760f33326","1":"1655702116","2":"56","_addr":"0x51967008fA944866Bb6Df61d1941E11760f33326","timestamp":"1655702116","res":"56"}
Account: 0x51967008fA944866Bb6Df61d1941E11760f33326
Balance before: 999960124915999988850
---> MyFunction called {"_events":{}}
Balance after: 999960124915999988850
Balance diff: 0


## 10.6 fallback 함수
## 실습문제 : fallback

In [3]:
%%writefile src/FallbackTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract FallbackTest {
    event PrintLog(string);
    function callA () pure public returns(string memory){
        return "doing callA";
    }
    fallback () external {
        emit PrintLog("fallback called");
    }
}

Overwriting src/FallbackTest.sol


In [4]:
!solc.exe --optimize --combined-json abi,bin src/FallbackTest.sol > src/FallbackTest.json

In [5]:
%%writefile src/FallbackTestDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./FallbackTest.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        });
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy();

Overwriting src/FallbackTestDeploy.js


In [6]:
!node src/FallbackTestDeploy.js

- contract name :  [ 'src/FallbackTest.sol:FallbackTest' ]
Deploying the contract from 0xc0463666BA29b481D7b33c630a4270F4f0dA581b
hash: 0x8fc4959e6883eaa127541093e1e8828fe23a7fdaf99912425468d1f0cdec545f
---> The contract deployed to: 0xDf67195d7AB4e49EC9CBaE893726382D6b996754


In [10]:
%%writefile src/FallbackTestUse.js
var Web3 = require('web3');
var _abiBinJson = require('./FallbackTest.json');
//var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before : " + balanceBefore);
    var _instance = new web3.eth.Contract(_abiArray, "0xDf67195d7AB4e49EC9CBaE893726382D6b996754");
    var event = _instance.events.PrintLog(function(error, result) {
        if(!error) {
            console.log("Event fired : " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
        }
    });
    
    _instance.methods.callA().call().then(function(res) {
        console.log(res);
    });
    web3.eth.sendTransaction({from: accounts[0], to: accounts[1]});
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after : " + balanceAfter);
    console.log("Balance diff : " + (balanceBefore - balanceAfter));
    process.exit(1);
}

doIt();

Overwriting src/FallbackTestUse.js


In [13]:
!node src/FallbackTestUse.js

- contract name :  [ 'src/FallbackTest.sol:FallbackTest' ]
Account: 0xc0463666BA29b481D7b33c630a4270F4f0dA581b
Balance before : 999999661070000000000
Balance after : 999999661070000000000
Balance diff : 0


## 실습문제 : 컨트랙 결합에서 payable fallback 사용

In [24]:
%%writefile src/MathMultiply7.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Multiply7 {
    receive() external payable {}

    fallback() external payable {}

    function multiply(uint input) pure public returns(uint) {
        return input * 7;
    }

    function getAddress() view public returns(address) {
        return address(this);
    }
}

contract Math {
    Multiply7 m7 = new Multiply7();

    function deposit(uint amount) payable public {
        require(msg.value == amount);
    }

    function setM7(address payable _addr) public {
        m7 = Multiply7(_addr);
    }

    function multiply() view public returns(uint) {
        uint res = m7.multiply(8);
        return res;
    }

    function send11M7() public payable {
        address(m7).transfer(11);
    }

    function getBalanceOfThis() public view returns(uint) {
        return address(this).balance;
    }

    function getBalanceOfM7() public view returns(uint) {
        return address(m7).balance;
    }

    function getAddressOfM7() view public returns(address) {
        return address(m7);
    }
}

Overwriting src/MathMultiply7.sol


In [31]:
!solc src/MathMultiply7.sol --combined-json abi > src/MathMultiply7ABI.json

In [32]:
!solc src/MathMultiply7.sol --combined-json bin > src/MathMultiply7BIN.json

In [41]:
%%writefile src/MathMultiply7Deploy.js
var Web3 = require('web3');
var _abiJson = require('./MathMultiply7ABI.json');
var _binJson = require('./MathMultiply7BIN.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractNames = Object.keys(_abiJson.contracts);
contractName = contractNames[0]; 
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi); 
_bin = _binJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
            if(!err) {
                console.log("hash : " + transactionHash);
            }
        });
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Overwriting src/MathMultiply7Deploy.js


In [42]:
!node src/MathMultiply7Deploy.js

- contract name :  src/MathMultiply7.sol:Math
Deploying the contract from 0xc0463666BA29b481D7b33c630a4270F4f0dA581b
hash : 0xd7e1447654c92f458fc9972d36516dd014528f2b6e94e45f9518d1d8104e0e76
---> The contract deployed to : 0x6ab70fbA5FCBD34b6B0c6BD84a4E6588A1AA709A


## 실습문제 : 인사하는 컨트랙

In [37]:
%%writefile src/Hello1.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Hello1 {
    string private hello;
    address private owner;
    event PrintLog(address addr, string s);

    constructor(string memory _hello) public {
        hello = _hello;
        owner = msg.sender;
    }

    function sayHello() view public returns(string memory) {
        return hello;
    }

    function setHello() public {
        string memory s = "";

        if(msg.sender == owner) {
            s = "Hello";
        }
        else {
            s = "olleh";
        }

        emit PrintLog(msg.sender, s);
        hello = s;
    }

    function compareTo(string memory _str) view public returns(bool) {
        bool isEqual = false;

        if(keccak256(bytes(hello)) == keccak256(bytes(_str))) {
            isEqual = true;
        }

        return isEqual;
    } 

    function getBalance() view public isOwner returns(uint) {
        return address(this).balance;
    }

    modifier isOwner() {
        if(msg.sender != owner) {
            revert();
        }
        _;
    }
}

Writing src/Hello1.sol


In [38]:
!solc.exe --optimize --combined-json abi,bin src/Hello1.sol > src/Hello1.json

In [1]:
%%writefile src/Hello1DeployFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./Hello1.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345")); 

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin, arguments: ["Hello from web3"]})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        });
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy();

Overwriting src/Hello1DeployFromFile.js


In [2]:
!node src/Hello1DeployFromFile.js

- contract name :  [ 'src/Hello1.sol:Hello1' ]
Deploying the contract from 0xc0463666BA29b481D7b33c630a4270F4f0dA581b
hash: 0x949d9f61dba740f1a0cd3f9dbb9ae579f7062097cbe6e801600f421cd676529a
---> The contract deployed to: 0xB8Bf79c527413E5Ac4Bb528411B8D671233b88A5


In [6]:
%%writefile src/Hello1UseFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./Hello1.json');
//var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345")); 

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function doIt() {
    var hello = new web3.eth.Contract(_abiArray, "0xB8Bf79c527413E5Ac4Bb528411B8D671233b88A5");
    var event = hello.events.PrintLog(function (error, event) {
        console.log(">>> Event fired: " + JSON.stringify(event.returnValues));
    })
    .on('>> data', function(event) {
        console.log(event);
    })
    .on('>> changed', function(event) {
        console.log(event);
    })
    .on('>> error', console.error);
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    hello.methods.sayHello().call().then(console.log);  //null
    await hello.methods.setHello().send({from: accounts[0]});
    hello.methods.sayHello().call().then(console.log);
    hello.methods.compareTo("Hello").call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    
}
doIt()

Overwriting src/Hello1UseFromFile.js


In [ ]:
!node src/Hello1UseFromFile.js 

# Q. 웹소켓을 사용하면 프로세스가 계속 되어서 인위적으로 해제해주어야 한다고 하는데 해제해주는 방법

## 실습문제 : 블록체인에서 컨트랙 삭제

In [7]:
%%writefile src/Hello2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Hello2 {
    string hello;
    address owner;

    event PrintLog(string _s);

    constructor() public {
        owner = msg.sender;
    }

    function sayHello() view public returns(string memory) {
        return hello;
    }

    function setHello(string memory _hello) public payable {
        hello = _hello;
        emit PrintLog(_hello);
    }

    function getBalance() view public returns(uint) {
        return address(this).balance;
    }

    function kill() public {
        if(msg.sender == owner) {
            selfdestruct(payable(owner));
        }
    }
}

Overwriting src/Hello2.sol


In [8]:
!solc.exe --optimize --combined-json abi,bin src/Hello2.sol > src/Hello2.json

In [9]:
%%writefile src/Hello2DeployFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./Hello2.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345")); 

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
            if(!err) {
                console.log("hash : " + transactionHash);
            }
        });
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Overwriting src/Hello2DeployFromFile.js


In [10]:
!node src/Hello2DeployFromFile.js

- contract name :  [ 'src/Hello2.sol:Hello2' ]
Deploying the contract from 0x4f0e664b54FAdF7AcF692374D1dC9b2a00c8fB7B
hash : 0x3049336a3df2115fe6f4fff0eb3d7510153e453a411a31d529525aee16688c43
---> The contract deployed to : 0x29DDEC2b79C09C1263dBD77Ab4e0F69C32caBce2


In [11]:
%%writefile src/Hello2UseFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./Hello2.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345")); 

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function doIt() {
    var hello = new web3.eth.Contract(_abiArray, "0x29DDEC2b79C09C1263dBD77Ab4e0F69C32caBce2");
    var event = hello.events.PrintLog(function(error, result) {
        if(!error) {
            console.log("Event fired : " + JSON.stringify(result.returnValues));
        }
    });
    const accounts = await web3.eth.getAccounts();
    console.log("Account : " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before : " + balanceBefore);
    hello.methods.sayHello().call().then(console.log);
    await hello.methods.setHello("Hello World").send({from: accounts[0], value: 1111});
    hello.methods.sayHello().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); 
}

doIt();

Writing src/Hello2UseFromFile.js


In [14]:
!node src/Hello2UseFromFile.js

- contract name :  [ 'src/Hello2.sol:Hello2' ]
Account : 0x4f0e664b54FAdF7AcF692374D1dC9b2a00c8fB7B
Balance before : 999999263825999997778
Hello World
Balance after: 999999206205999996667
Balance diff: 57620000014336


## 11.1.4 무작위 수의 생성

In [15]:
%%writefile src/Random.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Random {
    function rand() public view returns(bytes32) {
        return keccak256(abi.encodePacked(block.timestamp, block.difficulty));
    }

    function rand0and250() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty))) % 251);
    }

    function rand0and9() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty))) % 10);
    }

    function rand0and2() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty))) % 3);
    }

    function getRandomInteger() view public returns(uint8[] memory) {
        uint8[] memory r = new uint8[](10);
        for(uint i = 0; i < r.length; i++) {
            r[i] = uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty))) % 10);
        }
        return r;
    }
}

Writing src/Random.sol


In [17]:
!solc.exe src/Random.sol

Compiler run successful, no output requested.


## 실습문제 : 배열

In [1]:
%%writefile src/ArrayTest2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;
pragma experimental ABIEncoderV2;

contract ArrayTest2 {
    string[3] cities1 = ["Seoul", "Sydeny", "Tokyo"];
    string[] cities2 = new string[](2);
    string[] cities3;
    uint24[5] balance = [255, 65536, 95, 50, 1];
    int[5] mathMarks;
    uint[3][] marks = [[100, 80, 95], [20, 30, 40]];

    function setLocalDynamicArr() pure public returns(uint) {
        uint[] memory myArr = new uint[](3);
        myArr[0] = 11;
        myArr[1] = 12;

        return myArr.length;
    }

    function getDynamicArrMemory() pure public returns(uint[] memory) {
        uint[] memory num = new uint[](3);
        for(uint i = 0; i < num.length; i++) {
            num[i] = i;
        }
        return num;
    }

    function getStringDynamicArrMemory() pure public {
        string[2] memory places = ["9000", "Sydney"];
        places[0] = "Seoul";
    }

    // function getCities1_() view public returns(string[] memory) {
    //     return cities1[0];
    // }

    function getCities1Length() view public returns(uint) {
        return cities1.length;
    }

    function setCities23() public {
        string memory my = "Seoul";
        cities2[0] = "New York";
        cities2.push(my);
        cities2.push("Busan");
        cities2.push("New York");
        cities2.push("Beijing");
    }

    function getCites2() view public returns(string[] memory) {
        return cities2;
    }

    function setMathMarks() public {
        uint8[5] memory temp = [100, 60, 95, 50, 80];
        for(uint i = 0; i < temp.length; i++) {
            mathMarks[i] = int(int8(temp[i]));
        }
    }

    function getMathAbove70_() view public returns(int[] memory) {
        int[] memory mathAbove70;
        uint counter = 0;
        for(uint8 i = 0; i < mathMarks.length; i++) {
            if(mathMarks[i] > 70) {
                mathAbove70[counter] = mathMarks[i];
                counter++;
            }
        }
        return mathAbove70;
    }

    function getMathAbove70() view public returns(int[] memory) {
        uint8 counter = 0;
        uint8 lengthOfMathAbove70 = 0;
        for(uint8 i = 0; i < mathMarks.length; i++) {
            if(mathMarks[i] > 70) {
                counter++;
            }
        }
        lengthOfMathAbove70 = counter;
        int[] memory mathAbove70 = new int[](lengthOfMathAbove70);
        counter = 0;
        for(uint i = 0; i < mathMarks.length; i++) {
            if(mathMarks[i] > 70) {
                mathAbove70[counter] = mathMarks[i];
                counter++;
            }
        }
        return mathAbove70;
    }

    function updateMarks() public returns(uint[3][] memory){
        marks[0][0]=90;
        return marks;
    }
    function getMarksArr() view public returns(uint[3][] memory) {
        return marks;
    }
    function getMarksLength() view public returns(uint) {
        return marks.length;
    }
}

Overwriting src/ArrayTest2.sol


In [2]:
!solc.exe --optimize --combined-json abi,bin src/ArrayTest2.sol > src/ArrayTest2.json

In [3]:
%%writefile src/ArrayTest2Deploy.js
var Web3 = require('web3');
var _abiBinJson = require('./ArrayTest2.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345")); 

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from : " + accounts[0]);
    
    new web3.eth.Contract(_abiArray).deploy({data: "0x" + _bin})
        .estimateGas(function(err, gas) {
            if(!err) {
                console.log(">>> gas : " + gas);
            }
        });
    
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas: 1621137}, function(err, transactionHash) {
            if(!err) {
                console.log("hash : " + transactionHash);
            }
        });
    console.log("---> The contract deployed to " + deployed.options.address);
}

1deploy();

Overwriting src/ArrayTest2Deploy.js


In [4]:
!node src/ArrayTest2Deploy.js

- contract name :  [ 'src/ArrayTest2.sol:ArrayTest2' ]
Deploying the contract from : 0x608D8C98a9B5Ec35bd0e3133FEeeb3c695F56280
hash : 0x56161a10f4a15db6fb205eb79dcd119181f7143bb043a19920dd564617c97974
---> The contract deployed to 0x7Ff8304e0c2ED8259A3c41BE368B9a9269349feC
>>> gas : 907914


In [11]:
%%writefile src/ArrayTest2Use.js
var Web3 = require('web3');
var _abiBinJson = require('./ArrayTest2.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345")); 

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function doIt() {
    var arr = new web3.eth.Contract(_abiArray, "0x7Ff8304e0c2ED8259A3c41BE368B9a9269349feC");
    const accounts = await web3.eth.getAccounts();
    console.log("Account : " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before : " + balanceBefore);
    
    arr.methods.setCities23().estimateGas(function(err, gas) {
        if(!err) {
            console.log(">>> gas : " + gas);
        }
    });
    
    await arr.methods.setCities23().send({from: accounts[0], gas: 1333100});
    arr.methods.getCites2().call().then(console.log);
    
    arr.methods.setMathMarks().estimateGas(function(err, gas) {
        if(!err) {
            console.log(">>> gas : " + gas);
        }
    });
    
    await arr.methods.setMathMarks().send({from: accounts[0], gas: 1623540});
    arr.methods.getMarksArr().call().then(console.log);
    
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after : " + balanceAfter);
    console.log("Balance diff : " + (balanceBefore - balanceAfter));
}

doIt();

Overwriting src/ArrayTest2Use.js


In [12]:
!node src/ArrayTest2Use.js

- contract name :  [ 'src/ArrayTest2.sol:ArrayTest2' ]
Account : 0x608D8C98a9B5Ec35bd0e3133FEeeb3c695F56280
Balance before : 999997917552000000000
>>> gas : 139851
[ 'New York', '', 'Seoul', 'Busan', 'New York', 'Beijing' ]
>>> gas : 132659
Balance after : 999997372532000000000
Balance diff : 545019999944704
[ [ '100', '80', '95' ], [ '20', '30', '40' ] ]


In [13]:
!node src/ArrayTest2Use.js

- contract name :  [ 'src/ArrayTest2.sol:ArrayTest2' ]
Account : 0x608D8C98a9B5Ec35bd0e3133FEeeb3c695F56280
Balance before : 999997372532000000000
>>> gas : 122200
[
  'New York', '',
  'Seoul',    'Busan',
  'New York', 'Beijing',
  'Seoul',    'Busan',
  'New York', 'Beijing'
]
Balance after : 999997061814000000000
Balance diff : 310718000005120
[ [ '100', '80', '95' ], [ '20', '30', '40' ] ]
>>> gas : 33203


## 실습: Members 배열

In [15]:
%%writefile src/Members.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Members {
    address owner;
    event PrintAddr(address arg);

    struct Member {
        uint id;
        string name;
    }

    Member[] public memberArr;

    constructor() public {
        owner = msg.sender;
    }

    function del() public {
        delete memberArr;
    }

    function delOne(uint i) public {
        delete memberArr[i];
    }

    function add(uint id, string memory name) public {
        memberArr.push(Member(id, name));
    }

    function getMember(string memory who) view public returns(uint, string memory ) {
        uint _id;
        string memory _name;
        for(uint i = 0; i < memberArr.length; i++) {
            _name = memberArr[i].name;
            if(keccak256(abi.encodePacked(_name)) == keccak256(abi.encodePacked(who))) {
                _id = memberArr[i].id;
                _name = memberArr[i].name;
            }
        }
        return (_id, _name);
    }

    function compareStr(string memory s1, string memory s2) pure public returns(bool) {
        return keccak256(abi.encodePacked(s1))==keccak256(abi.encodePacked(s2));
    }

    function compareBytes(bytes memory b1, bytes memory b2) pure public returns(bool) {
        return keccak256(b1) == keccak256(b2);
    }

    function getLength() view public returns(uint) {
        return memberArr.length;
    }
}





Writing src/Members.sol


In [16]:
!solc.exe src/Members.sol

Compiler run successful, no output requested.


## 11.2.2 mapping

In [17]:
%%writefile src/BankV4.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract BankV4 {
    address owner;
    mapping (address => uint) balanceOf;

    constructor() public {
        owner = msg.sender;
    }

    function deposit(uint amount) payable public onlyOwner {
        require(msg.value == amount);
        balanceOf[msg.sender] += amount;
    }

    function forwardTo(address receiver, uint amount) payable public onlyOwner {
        require(balanceOf[msg.sender] >= amount);
        balanceOf[msg.sender] -= amount;
        balanceOf[receiver] += amount;
    }

    function withdraw(address payable receiver, uint amount) public onlyOwner {
        require(balanceOf[receiver] >= amount && address(this).balance >= amount);
        balanceOf[receiver] -= amount;
        receiver.transfer(amount);
    }

    function getBalance() public view returns(uint, uint) {
        return(address(this).balance, balanceOf[owner]);
    }

    function getBalanceOf(address addr) public view returns(uint) {
        return balanceOf[addr];
    }

    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
 }


Overwriting src/BankV4.sol


In [18]:
!solc.exe src/BankV4.sol

Compiler run successful, no output requested.


In [19]:
%%writefile src/MembersMap.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract MembersMap {
    struct Member {
        uint id;
        string name;
    }

    mapping(address => Member) memberMap;
    mapping(uint => address) addressById;
    mapping(string => address) addressByName;

    function addMember(uint _id, string memory _name) public {
        Member memory m = Member(_id, _name);
        memberMap[msg.sender] = m;
        addressById[_id] = msg.sender;
        addressByName[_name] = msg.sender;
    }

    function getMemberById(uint _id) view public returns(uint, string memory) {
        Member memory my = memberMap[addressById[_id]];
        return (my.id, my.name);
    }

    function getMemberByName(string memory _name) public view returns(address) {
        return addressByName[_name];
    }

    function getMember(address addr) view public returns(uint, string memory) {
        Member memory m = memberMap[addr];
        return (m.id, m.name);
    }
}

Writing src/MembersMap.sol


In [20]:
!solc.exe src/MembersMap.sol

Compiler run successful, no output requested.


## 11.3 예외
## 실습문제 : 예외

In [21]:
%%writefile src/ExceptionTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract ExceptionTest {
    address owner;
    
    constructor() public {
        owner = msg.sender;
    }

    function requireException() view public returns(string memory) {
        require(msg.sender != owner, "Sorry! You are owner. Require failed...");
        return "require conditon met";
    }

    function assertException() view public returns(string memory) {
        assert(msg.sender == owner);
        return "asserted";
    }

    function revertException() view public returns(string memory) {
        if(msg.sender != owner) {
            revert("Sorry! You are NOT owner. Reverted...");
        }
        else {
            return "not reverted";
        }
    }

    function raiseException() pure public {
        int x = 0;
        int y = 0;
        x / y;
    }
}


Writing src/ExceptionTest.sol


In [22]:
!solc.exe src/ExceptionTest.sol

Compiler run successful, no output requested.


## 연습문제 : 주문

## 12.1.1 컨트랙 간의 관련
## 실습 문제: new 명령어로 컨트랙 조립 composition

In [1]:
%%writefile src/C1C2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract C1 {
    uint128 v1;
    
    function set(uint128 _v1) public {
        v1 = _v1;
    }

    function get() public view returns(uint128) {
        return v1;
    }

    function get7() public pure returns(uint128) {
        return 7;
    }
}

contract C2 {
    C1 c1;

    constructor() {
        c1 = new C1();
    }

    function set(uint128 _v1) public {
        c1.set(_v1);
    }

    function get() public view returns(uint128) {
        return c1.get();
    }

    function get7() public view returns(uint128) {
        return c1.get7();
    }

    function getC1Address() public view returns(address) {
        return address(c1);
    }
}

Writing src/C1C2.sol


In [2]:
!solc-windows.exe src/C1C2.sol --combined-json abi,bin > src/C1C2.json

In [7]:
%%writefile src/C1C2Deploy.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1C2.json");
var _json=JSON.parse(_str)
var _abiArray=_json.contracts["src/C1C2.sol:C2"].abi;
var _bin="0x"+_json.contracts["src/C1C2.sol:C2"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash"+hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR "+error);
        });
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/C1C2Deploy.js


In [8]:
!node src/C1C2Deploy.js

Deploying the contract from 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
>>> transactionHash0xba9c61de094729f822542d97357d3dce92d242780b8c88aa17a6e53fb4296295
>>> RECEPIT hash: 0xba9c61de094729f822542d97357d3dce92d242780b8c88aa17a6e53fb4296295
>>> address:0x83bbd058d05D4F7439F295dd0e9b430CBfEbEAea
---> The contract deployed to: 0x83bbd058d05D4F7439F295dd0e9b430CBfEbEAea


In [11]:
%%writefile src/C1C2Use.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1C2.json");
var _json=JSON.parse(_str);
var _abiArray=_json.contracts["src/C1C2.sol:C2"].abi;

var c2 = new web3.eth.Contract(_abiArray, "0x83bbd058d05D4F7439F295dd0e9b430CBfEbEAea");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    c2.methods.get7().call().then(function(res) {console.log("C1 get7(): " + res)});
    await c2.methods.set(9).send({from: accounts[0],gas:50000}, function(err,res) {
        console.log("setting 9..."+res);
    });
    c2.methods.get().call().then(function(res) {console.log("C1 get(): " + res)});
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/C1C2Use.js


In [12]:
!node src/C1C2Use.js

Account: 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
Balance before: 999999013462000000000
C1 get7(): 7
setting 9...0xb64a9e74a28da70d85bf09e0e37dc69e63815754beb5c9ce8122587ac2caf0d9
Balance after: 999998954606000000000
Balance diff: 58856000061440
C1 get(): 9


## 실습: 컨트랙의 주소를 사용하여 컨트랙 결합 association

In [13]:
%%writefile src/C1.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract C1 {
    uint128 v1;

    function set(uint128 _v1) public {
        v1 = _v1;
    }

    function get() public view returns(uint128) {
        return v1;
    }

    function get7() public pure returns(uint128) {
        return 7;
    }
}

Overwriting src/C1.sol


In [14]:
!solc-windows.exe src/C1.sol --combined-json abi,bin > src/C1.json

In [17]:
%%writefile src/C1Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs = require('fs');
var _str = fs.readFileSync("src/C1.json");
var _json = JSON.parse(_str);
var _abiArray = _json.contracts["src/C1.sol:C1"].abi;
var _bin = "0x" + _json.contracts["src/C1.sol:C1"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from : " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash) {
            console.log(">>> transaction hash : " + hash);
        })
        .on('receipt', function(receipt) {
            console.log(">>> receipt hash : " + receipt.transactionHash + "\n>>> address : " + receipt.contractAddress);
        })
        .on('error', function(error, result) {
            console.log(">>> error : " + error);
        });
    
    console.log("---> The contract deployed to: " + deployed.options.address);
}

deploy();

Overwriting src/C1Deploy.js


In [18]:
!node src/C1Deploy.js

Deploying the contract from : 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
>>> transaction hash : 0x64bcfc5bb5068f397d56f090151a04bca077d46f452855048e43721f810f3c5a
>>> receipt hash : 0x64bcfc5bb5068f397d56f090151a04bca077d46f452855048e43721f810f3c5a
>>> address : 0xF450C18b10838E653263b48F37214e3D19e73076
---> The contract deployed to: 0xF450C18b10838E653263b48F37214e3D19e73076


In [19]:
%%writefile src/C1Use.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1.json");
var _json=JSON.parse(_str);
var _abiArray = _json.contracts["src/C1.sol:C1"].abi;
var c1 = new web3.eth.Contract(_abiArray, "0xF450C18b10838E653263b48F37214e3D19e73076");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account : " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before : " + balanceBefore);
    c1.methods.get7().call().then(console.log);
    await c1.methods.set(9).send({from: accounts[0], gas: 50000});
    c1.methods.get().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after : " + balanceAfter);
    console.log("Balance diff : " + (balanceBefore - balanceAfter));
}
doIt();

Overwriting src/C1Use.js


In [20]:
!node src/C1Use.js

Account : 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
Balance before : 999998644332000000000
7
Balance after : 999998556766000000000
Balance diff : 87566000062464
9


In [22]:
%%writefile src/C2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
import "./C1.sol";

contract C2 {
    C1 c1;

    constructor() {
        c1 = new C1();
    }

    function setC1(address _addressOfC1) public {
        c1 = C1(_addressOfC1);
    }

    function set(uint128 _v1) public {
        c1.set(_v1);
    }

    function get() public view returns(uint128) {
        return c1.get();
    }

    function get7() public view returns(uint128) {
        return c1.get7();
    }

    function getC1Address() public view returns(address) {
        return address(c1);
    }
}

Overwriting src/C2.sol


In [23]:
!solc-windows.exe src/C2.sol --combined-json abi,bin > src/C2.json

In [24]:
%%writefile src/C2Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C2.json");
var _json = JSON.parse(_str);
var _abiArray = _json.contracts["src/C1.sol:C1"].abi;
var _bin = "0x"+_json.contracts["src/C2.sol:C2"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash" + hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR " + error);
        });
    console.log("---> The contract deployed to: " + deployed.options.address);
}

deploy();

Overwriting src/C2Deploy.js


In [25]:
!node src/C2Deploy.js

Deploying the contract from 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
>>> transactionHash0x30e344547bec2fc0e3dd60de2cbcb48cb3874dd62d8c51da4c44eade88812da8
>>> RECEPIT hash: 0x30e344547bec2fc0e3dd60de2cbcb48cb3874dd62d8c51da4c44eade88812da8
>>> address:0x07ED7599B0d693E190eB2d4a0f06518F53e64AaA
---> The contract deployed to: 0x07ED7599B0d693E190eB2d4a0f06518F53e64AaA


In [28]:
%%writefile src/C2Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C2.json");
var _json = JSON.parse(_str);
var _abiArray = _json.contracts["src/C2.sol:C2"].abi;

var c2 = new web3.eth.Contract(_abiArray, "0x07ED7599B0d693E190eB2d4a0f06518F53e64AaA");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account : " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before : " + balanceBefore);
    console.log("--- new C1 ---");
    await c2.methods.getC1Address().call(function(err, c1addr) {
        if(!err) {
            console.log("c1 address by 'new' : " + c1addr);
        }
    });
    c2.methods.get7().call().then(function(res) { console.log("get7(): "+res) });
    console.log("--- set the above deployed address of C1 ---");
    await c2.methods.setC1("0xF450C18b10838E653263b48F37214e3D19e73076").send({from:accounts[0], gas:50000});
    await c2.methods.getC1Address().call(function(err, c1addr) {
        if(!err) console.log("c1 address by 'setC1()': "+c1addr);
    });
    c2.methods.get7().call().then(console.log);
    await c2.methods.set(222).send({from: accounts[0],gas:50000});
    c2.methods.get().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt();

Overwriting src/C2Use.js


In [29]:
!node src/C2Use.js

Account : 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
Balance before : 999997586562000000000
--- new C1 ---
c1 address by 'new' : 0x4aFC48bfC5a7C4D2F31C1009EC0a0d57dA8B51a8
--- set the above deployed address of C1 ---
get7(): 7
c1 address by 'setC1()': 0xF450C18b10838E653263b48F37214e3D19e73076
7
Balance after: 999997468110000000000
Balance diff: 118451999997952
222


In [30]:
!node src/C2Use.js

Account : 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
Balance before : 999997468110000000000
--- new C1 ---
c1 address by 'new' : 0xF450C18b10838E653263b48F37214e3D19e73076
--- set the above deployed address of C1 ---
get7(): 7
c1 address by 'setC1()': 0xF450C18b10838E653263b48F37214e3D19e73076
7
Balance after: 999997360858000000000
Balance diff: 107251999965184
222


## 실습 문제 : 자동차와 엔진의 조립 - new 명령어 사용

In [31]:
%%writefile src/Car.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Car {
    Engine engineObj;
    string private color;
    event PrintLog(address sender, string msg);

    constructor() {
        engineObj = new Engine();
    }

    function setColor(string memory _color) public {
        color = _color;
    }

    function getColor() public view returns(string memory) {
        return color;
    }
    
    function getSpeed() public view returns(uint) {
        return engineObj.getSpeed();
    }

    function speedUpBy10() public {
        engineObj.setSpeendUpBy(10);
    }

    function speedDownBy10() public {
        engineObj.setSpeedDownBy(10);
    }

    function start() public {
        engineObj.on();
        string memory engineStateStr = engineObj.getEngineState() ? "on" : "off";
        emit PrintLog(msg.sender, engineStateStr);
    }
}

contract Engine {
    uint constant private MAXSPEED = 200;
    uint private speed;
    bool private engineState;

    constructor() {
        speed = 0;
        off();
    }

    function on() public {
        engineState = true;
    }

    function off() public {
        engineState = false;
    }

    function getEngineState() public view returns(bool) {
        return engineState;
    }

    function setSpeendUpBy(uint _speed) public {
        if(speed < (MAXSPEED - 10) && engineState == true)
            speed += _speed;
    }

    function setSpeedDownBy(uint _speed) public {
        if((speed - 10) > 0 && engineState == true)
            speed -= _speed;
    }

    function getSpeed() public view returns(uint) {
        return speed;
    }
}

Writing src/Car.sol


In [32]:
!solc-windows.exe src/Car.sol --combined-json abi,bin > src/Car.json

In [33]:
%%writefile src/carDeploy.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Car.json");
var _json=JSON.parse(_str);
var _abiArray=_json.contracts["src/Car.sol:Car"].abi;
var _bin="0x"+_json.contracts["src/Car.sol:Car"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash"+hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR "+error);
        });
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy();

Writing src/carDeploy.js


In [34]:
!node src/carDeploy.js

Deploying the contract from 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
>>> transactionHash0x650bd0fb26909ba72a832d21f348fbd4be209b94b4070ec70feeac6583da3158
>>> RECEPIT hash: 0x650bd0fb26909ba72a832d21f348fbd4be209b94b4070ec70feeac6583da3158
>>> address:0x44e1a1FEbb8789FF90DD4EC2cBd9adc7D53AAdF2
---> The contract deployed to: 0x44e1a1FEbb8789FF90DD4EC2cBd9adc7D53AAdF2


In [35]:
%%writefile src/carUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Car.json");
var _json=JSON.parse(_str);
var _abiArray=_json.contracts["src/Car.sol:Car"].abi;

async function doIt() {
    var car = new web3.eth.Contract(_abiArray, "0x44e1a1FEbb8789FF90DD4EC2cBd9adc7D53AAdF2");
    car.events.PrintLog({fromBlock: 'latest', toBlock:'pending'}, function (error, event) {
            console.log(">>> Event fired: " + JSON.stringify(event.returnValues));
        }).on('>> data', function(event){
            console.log(event); // same results as the optional callback above
        }).on('>> changed', function(event){
            console.log(event); // remove event from local database
        }).on('>> error', console.error);

    var speed;
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    car.methods.setColor("RED").send({from: accounts[0], gas:100000});
    car.methods.getColor().call().then(console.log);
    await car.methods.start().send({from: accounts[0], gas:100000})
    await car.methods.speedUpBy10().send({from: accounts[0], gas:100000})
    car.methods.getSpeed().call().then(function(speed) { console.log("-> speed: " + speed) });
    await car.methods.start().send({from: accounts[0], gas:100000})
    await car.methods.speedUpBy10().send({from: accounts[0], gas:100000})
    car.methods.getSpeed().call().then(function(speed) { console.log("-> speed: " + speed) });
    await car.methods.speedDownBy10().send({from: accounts[0], gas:100000})
    car.methods.getSpeed().call().then(function(speed) { console.log("-> speed: " + speed) });
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); //force exit -> may terminate some functions (speedDownBy10, getSpeed)
}

doIt()

Writing src/carUse.js


In [36]:
!node src/carUse.js

Account: 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
Balance before: 999995621524000000000

>>> Event fired: {"0":"0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff","1":"on","sender":"0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff","msg":"on"}
-> speed: 10
>>> Event fired: {"0":"0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff","1":"on","sender":"0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff","msg":"on"}
-> speed: 20
Balance after: 999995120166000000000
Balance diff: 501357999947776


## 실습 문제 : 사각형과 면적의 조립 - new 명령어로 사각형 생성하고 주소 출력

In [37]:
%%writefile src/SquareArea.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Square {
    uint128 length;
    
    function getLength() public view returns(uint128) {
        return length;
    }

    function setLength(uint128 _length) public {
        length = _length;
    }
    
    function getDegree() public pure returns(uint128) {
        return 90;
    }
}

contract Area {
    Square s;
    address owner;

    constructor() {
        s = new Square();
        owner = msg.sender;
    }

    function changeSquare(address _addressOfSquare) public {
        s = Square(_addressOfSquare);
    }

    function calcArea() view public returns(uint128) {
        uint128 length = s.getLength();
        uint128 area = length * length;
        return area;
    }

    function setLength(uint128 _length) public {
        s.setLength(_length);
    }

    function getLength() public view returns(uint128) {
        return s.getLength();
    }

    function getDegree() public view returns(uint128) {
        return s.getDegree();
    }

    function getAddressOfSquare() public view returns(address) {
        return address(s);
    }
}

Overwriting src/SquareArea.sol


In [38]:
!solc-windows.exe src/SquareArea.sol --combined-json abi,bin > src/SquareArea.json

In [39]:
%%writefile src/SquareAreaDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/SquareArea.json");
var _json=JSON.parse(_str)
var _abiArray = _json.contracts["src/SquareArea.sol:Area"].abi;
var _bin = "0x" + _json.contracts["src/SquareArea.sol:Area"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash"+hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR "+error);
        });
    console.log("---> The contract deployed to: " + deployed.options.address);
}
deploy();

Overwriting src/SquareAreaDeploy.js


In [40]:
!node src/SquareAreaDeploy.js

Deploying the contract from 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
>>> transactionHash0x79d154ad8f63e8f2d62f6ad9bad8e41e3d0c272db2ea9e78803d296e6f79f6b5
>>> RECEPIT hash: 0x79d154ad8f63e8f2d62f6ad9bad8e41e3d0c272db2ea9e78803d296e6f79f6b5
>>> address:0x214F90FdfAC254BC5e89b840aF0bA22B9fAeB266
---> The contract deployed to: 0x214F90FdfAC254BC5e89b840aF0bA22B9fAeB266


In [45]:
%%writefile src/SquareAreaUse.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/SquareArea.json");
var _json = JSON.parse(_str)
var _abiArray = _json.contracts["src/SquareArea.sol:Area"].abi;

async function doIt() {
    var area = new web3.eth.Contract(_abiArray, "0x214F90FdfAC254BC5e89b840aF0bA22B9fAeB266");
    var speed;
    const accounts = await web3.eth.getAccounts();
    console.log("Account : " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before : " + balanceBefore);
    area.methods.getDegree().call().then(console.log);
    await area.methods.setLength(9).send({from: accounts[0]});
    area.methods.getLength().call().then(console.log);
    area.methods.calcArea().call().then(console.log);
    area.methods.getAddressOfSquare().call().then(function(address) {
        console.log("Square Address : " + address);
    });
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after : " + balanceAfter);
    console.log("Balance diff : " + (balanceBefore - balanceAfter));
}

doIt();

Overwriting src/SquareAreaUse.js


In [46]:
!node src/SquareAreaUse.js

^C


## 실습 문제 : 사각형과 면적의 조립 - import문으로 사각형 포함, 이미 배포된 사각형 컨트랙으로 교체해보기

In [47]:
%%writefile src/Square.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Square {
    uint128 length;

    function getLength() public view returns(uint128) {
        return length;
    }

    function setLength(uint128 _length) public {
        length = _length;
    }

    function getDegree() public pure returns(uint128) {
        return 90;
    }
}

Overwriting src/Square.sol


In [48]:
%%writefile src/Area.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
import "./Square.sol";

contract Area {
    Square s;
    address owner;

    constructor() {
        s = new Square();
        owner = msg.sender;
    }

    function changeSquare(address _addressOfSquare) public {
        s = Square(_addressOfSquare);
    }

    function calcArea() view public returns(uint128) {
        uint128 length = s.getLength();
        uint128 area = length * length;
        return area;
    }

    function setLength(uint128 _length) public {
        s.setLength(_length);
    }

    function getLength() public view returns(uint128) {
        return s.getLength();
    }

    function getDegree() public view returns(uint128) {
        return s.getDegree();
    }

    function getAddressOfSquare() public view returns(address) {
        return address(s);
    }
}


Overwriting src/Area.sol


In [49]:
!solc-windows.exe src/Area.sol --combined-json abi,bin > src/Area.json

In [53]:
%%writefile src/AreaDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs = require('fs');
var _str = fs.readFileSync("src/Area.json");
var _json = JSON.parse(_str)
var _abiArray = _json.contracts["src/Area.sol:Area"].abi;
var _bin = "0x" + _json.contracts["src/Area.sol:Area"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from : " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
            if(!err) console.log("hash: " + transactionHash);
        });
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Overwriting src/AreaDeploy.js


In [54]:
!node src/AreaDeploy.js

Deploying the contract from : 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
hash: 0xde47df424749fc12d075215622c28689452069fad33466c9aa9d454177fabbf9
---> The contract deployed to : 0x24B3E9A7C8acC6aA3133e8BaC8F49B8bd0489186


In [57]:
%%writefile src/AreaUse.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));      
var fs=require('fs');
var _str = fs.readFileSync("src/Area.json");
var _json = JSON.parse(_str)
var _abiArray = _json.contracts["src/Area.sol:Area"].abi;

async function doIt() {
    var hello = new web3.eth.Contract(_abiArray, "0x24B3E9A7C8acC6aA3133e8BaC8F49B8bd0489186");
    const accounts = await web3.eth.getAccounts();
    console.log("Account : " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before : " + balanceBefore);
    hello.methods.getAddressOfSquare().call(function(err, c1addr) {
        if(!err) console.log(">>> Square address by 'new' : " + c1addr);
    });
    await hello.methods.setLength(10).send({from: accounts[0]});
    hello.methods.getLength().call().then(console.log);
    hello.methods.calcArea().call().then(console.log);
    hello.methods.getDegree().call().then(console.log);
    hello.methods.getAddressOfSquare().call().then(console.log);
    
    await hello.methods.changeSquare('0x214F90FdfAC254BC5e89b840aF0bA22B9fAeB266')
        .send({from: accounts[0]});
    hello.methods.getAddressOfSquare().call(function(err, c1addr) {
        if(!err) console.log(">>> Square address by 'changeSquare'" + c1addr);
    });
    
    await hello.methods.setLength(10).send({from: accounts[0]});
    hello.methods.getLength().call().then(console.log);
    hello.methods.calcArea().call().then(console.log);
    hello.methods.getDegree().call().then(console.log);
    hello.methods.getAddressOfSquare().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after : " + balanceAfter);
    console.log("Balance diff : " + (balanceBefore - balanceAfter));
    
}

doIt();

Overwriting src/AreaUse.js


In [58]:
!node src/AreaUse.js

Account : 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
Balance before : 999992401028000000000
>>> Square address by 'new' : 0x214F90FdfAC254BC5e89b840aF0bA22B9fAeB266
0x214F90FdfAC254BC5e89b840aF0bA22B9fAeB266
90
10
100
>>> Square address by 'changeSquare'0x214F90FdfAC254BC5e89b840aF0bA22B9fAeB266
Balance after : 999992212476000000000
Balance diff : 188552000045056
0x214F90FdfAC254BC5e89b840aF0bA22B9fAeB266
90
10
100


## 12.2.1 상속 is-a 관계
## 실습문제 : 점과 사각형의 상속

In [59]:
%%writefile src/Point.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Point {
    int x;
    int y;

    constructor(int _x, int _y) {
        x = _x;
        y = _y;
    }

    function getX() view public returns(int) {
        return x;
    }

    function getY() view public returns(int) {
        return y;
    }
}

Writing src/Point.sol


In [60]:
%%writefile src/Rectangle.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
import "./Point.sol";

contract Rectangle is Point {
    int w;
    int h;

    constructor(int _w, int _h, int _x, int _y) Point(_x, _y) {
        w = _w;
        h = _h;
    }

    function getPerimeter() view public returns(int) {
        return 2 * (w + h);
    }

    function getXOpposite() view public returns(int) {
        return getX() + w;
    }

    function getYOpposite() view public returns(int) {
        return getY() + h;
    }
}

Writing src/Rectangle.sol


In [61]:
!solc-windows.exe src/Rectangle.sol --combined-json abi,bin > src/Rectangle.json

In [62]:
%%writefile src/RectangleDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs = require('fs');
var _str = fs.readFileSync("src/Rectangle.json");
var _json = JSON.parse(_str)
var _abiArray = _json.contracts["src/Rectangle.sol:Rectangle"].abi;
var _bin = "0x" + _json.contracts["src/Rectangle.sol:Rectangle"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from : " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin, arguments: [10, 20, 30, 40]})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
            if(!err) console.log("hash : " + transactionHash);
        });
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Writing src/RectangleDeploy.js


In [63]:
!node src/RectangleDeploy.js

Deploying the contract from : 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
hash : 0x3680d22f608d50200811fe634ec4de290f4820027a39379169ecb5522900ca25
---> The contract deployed to : 0x5F7C416BE008B91950a1417628Ba668db825B955


In [64]:
%%writefile src/RectangleUse.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));     
var fs = require('fs');
var _str = fs.readFileSync("src/Rectangle.json");
var _json = JSON.parse(_str)
var _abiArray = _json.contracts["src/Rectangle.sol:Rectangle"].abi;

async function doIt() {
    var rect = new web3.eth.Contract(_abiArray, "0x5F7C416BE008B91950a1417628Ba668db825B955");
    const accounts = await web3.eth.getAccounts();
    console.log("(1) Account : " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("(2) Balance before : " + balanceBefore);
    rect.methods.getPerimeter().call().then(function(res) {
        console.log("(3) Perimeter: "+res)
    });
    rect.methods.getXOpposite().call().then(function(res) {
        console.log("(4) X opp: "+res)
    });
    rect.methods.getYOpposite().call().then(function(res) {
        console.log("(5) Y opp: "+res)
    });
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("(6) Balance after : " + balanceAfter);
    console.log("(7) Balance diff : " + (balanceBefore - balanceAfter));
}
  
doIt();

Writing src/RectangleUse.js


In [65]:
!node src/RectangleUse.js

(1) Account : 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
(2) Balance before : 999991520082000000000
(6) Balance after : 999991520082000000000
(7) Balance diff : 0
(3) Perimeter: 60
(4) X opp: 40
(5) Y opp: 60


## 실습 문제 : greeter.sol

In [66]:
%%writefile src/greeter.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract mortal {
    address owner;

    constructor() {
        owner = msg.sender;
    }

    function kill() public {
        if(msg.sender == owner) {
            selfdestruct(payable(owner));
        }
    }
}

contract greeter is mortal {
    string greeting;

    constructor(string memory _greeting) {
        greeting = _greeting;
    }

    function greet() public view returns(string memory) {
        return greeting;
    }

    function getOwner() view public returns(address) {
        return owner;
    }

}

Overwriting src/greeter.sol


In [67]:
!solc-windows.exe src/greeter.sol --combined-json abi,bin > src/greeter.json

In [70]:
%%writefile src/greeterDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./greeter.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); 
console.log("- contract name: ", contractName);
_abiArray=_abiBinJson.contracts[contractName[0]].abi;
_bin="0x"+_abiBinJson.contracts[contractName[0]].bin;

var _contract = new web3.eth.Contract(_abiArray);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from : " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin, arguments: ['Hello jsl']})
        .send({from : accounts[0], gas: 400000}, function(err, transactionHash) {
            if(!err) console.log("hash : " + transactionHash);
        });
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Overwriting src/greeterDeploy.js


In [71]:
!node src/greeterDeploy.js

- contract name:  [ 'src/greeter.sol:greeter', 'src/greeter.sol:mortal' ]
Deploying the contract from : 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
hash : 0x2c3acfad7a742d418da7d915e521ed57ca71e57c2781f36f1f5ca99562ef3ab2
---> The contract deployed to : 0x28dcFE1e8CFD47B9775B90AbA75E302602d0dc67


In [73]:
%%writefile src/greeterUse.js
var Web3 = require('web3');
var _abiBinJson = require('./greeter.json');      //importing a javascript file
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); 
console.log("- contract name: ", contractName);
console.log(contractName[0], contractName[1]); // index 0 -> ['src/greeter.sol:greeter']
_abiArray=_abiBinJson.contracts[contractName[0]].abi;
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
var greet = new web3.eth.Contract(_abiArray, '0x28dcFE1e8CFD47B9775B90AbA75E302602d0dc67');

async function doIt() {
    greet.methods.greet().call().then(function(res) {
        console.log("greet : " + res);
    });
    greet.methods.getOwner().call().then(function(res) {
        console.log("owner : " + res);
    });
}

doIt();

Overwriting src/greeterUse.js


In [74]:
!node src/greeterUse.js

- contract name:  [ 'src/greeter.sol:greeter', 'src/greeter.sol:mortal' ]
src/greeter.sol:greeter src/greeter.sol:mortal
owner : 0x27cDD04B660Aa736c6e8ad2527467320A38B14Ff
greet : Hello jsl


## 13.1.5 디파이 개발

In [1]:
%%writefile src/MyToken.sol
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "../node_modules/@openzeppelin/contracts/token/ERC20/ERC20.sol";
//import "@openzeppelin/contracts/token/ERC20/ERC20.sol";

contract MyToken is ERC20 {
    constructor() ERC20("MyToken", "JSL"){
        _mint(msg.sender, 1000000000000000000000000);
        //_mint(msg.sender, 1000000*decimals());
    }
}

Overwriting src/MyToken.sol


In [2]:
!solc-windows.exe @openzeppelin/="C:\\Users\\youve\\Code\\201810820\\node_modules\\@openzeppelin\\" --optimize --combined-json abi,bin src/MyToken.sol > src/MyToken.json

In [3]:
%%writefile src/MyTokenDeployPractice.js
var Web3=require('web3');
var _abiBinJson = require('./MyToken.json');

var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

contractName=Object.keys(_abiBinJson.contracts);
console.log("- contract name: ", contractName[4]); //or console.log(contractName);
_abiArray=JSON.parse(JSON.stringify(_abiBinJson.contracts[contractName[4]].abi));    //JSON parsing needed!!
_bin="0x"+_abiBinJson.contracts[contractName[4]].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from : accounts[0], gas: 1000000}, function(err, transactionHash) {
            if(!err) console.log("hash : " + transactionHash);
        });
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Writing src/MyTokenDeployPractice.js


In [4]:
!node src/MyTokenDeployPractice.js

- contract name:  src/MyToken.sol:MyToken
Deploying the contract from 0x41fE5792b2FC66e84BE3e732d38b5dcDfD7834b5
hash : 0x6c63b2f1c4f8ea68b2e5b04c311e795ecdffe602f90b97d26461fdb655a9ac7c
---> The contract deployed to : 0x0509faC64FA04A847c06113a8Ed743048f4Ec78E


In [5]:
%%writefile src/MyTokenUsePractice.js
var Web3=require('web3');
var _abiBinJson = require('./MyToken.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); 
console.log("- contract name: ", contractName[4]);
var _abiArray=JSON.parse(JSON.stringify(_abiBinJson.contracts[contractName[4]].abi));
var _test = new web3.eth.Contract(_abiArray, '0x0509faC64FA04A847c06113a8Ed743048f4Ec78E');

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account : " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before : " + balanceBefore);
    
    const name = await _test.methods.name().call();
    console.log("name : " + name);
    const decimals = await _test.methods.decimals().call();
    console.log("decimals : " + decimals);
    const values = await _test.methods.totalSupply().call();
    console.log("totalsupply : " + values);
    
    console.log("-----1 before transfer-----");
    console.log("1-1 balanceOfAccount0: " + await _test.methods.balanceOf(accounts[0]).call());
    console.log("1-2 balanceOfAccount1: " + await _test.methods.balanceOf(accounts[1]).call());
    await _test.methods.transfer(accounts[1], 1)
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
    console.log("-----2 after transfer-----");
    console.log("2-1 balanceOfAccount0: " + await _test.methods.balanceOf(accounts[0]).call());
    console.log("2-2 balanceOfAccount1: " + await _test.methods.balanceOf(accounts[1]).call());
    console.log("-----3 after approve and transferFrom-----");    
    var addrTo = accounts[0];
    console.log("3-1 allowanceOfAddrTo: "+ await _test.methods.allowance(accounts[0], addrTo).call());
    await _test.methods.approve(addrTo, 111)
        .send({from: accounts[0], gas: 100000});
    console.log("3-2 allowanceOfAddrTo 111: "+ await _test.methods.allowance(accounts[0], addrTo).call());
    await _test.methods.transferFrom(addrTo, accounts[1], 1)
        .send({from: accounts[0], gas: 3641240, gasPrice: '1000000000'})
    console.log("3-3 balanceOfAccount0: " + await _test.methods.balanceOf(accounts[0]).call());
    console.log("3-4 balanceOfAccount1: " + await _test.methods.balanceOf(accounts[1]).call());
    //const value = await _test.methods.myFunction()
    //    .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    //console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt();

Writing src/MyTokenUsePractice.js


In [6]:
!node src/MyTokenUsePractice.js

- contract name:  src/MyToken.sol:MyToken
Account : 0x41fE5792b2FC66e84BE3e732d38b5dcDfD7834b5
Balance before : 999998668648000000000
name : MyToken
decimals : 18
totalsupply : 1000000000000000000000000
-----1 before transfer-----
1-1 balanceOfAccount0: 1000000000000000000000000
1-2 balanceOfAccount1: 0
-----2 after transfer-----
2-1 balanceOfAccount0: 999999999999999999999999
2-2 balanceOfAccount1: 1
-----3 after approve and transferFrom-----
3-1 allowanceOfAddrTo: 0
3-2 allowanceOfAddrTo 111: 111
3-3 balanceOfAccount0: 999999999999999999999998
3-4 balanceOfAccount1: 2
Balance after: 999998481929000000000
Balance diff: 186718999937024


In [ ]:
https://ipfs.io/ipfs/QmZSMw1PARg1dDxaN2CoNbskraRXJo7trUsoUickrRgrTn

## 13.2.6 NFT 개발

In [7]:
%%writefile src/MyNFT.sol
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC721/ERC721.sol";
//import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/v4.4.0/contracts/token/ERC721/ERC721.sol";
import "@openzeppelin/contracts/utils/Counters.sol";
//import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/v4.4.0/contracts/utils/Counters.sol";

contract MyNFT is ERC721 {
    using Counters for Counters.Counter;
    Counters.Counter private _tokenIds;	//NFT id
    
    constructor() ERC721("Hello DApp Screens", "JSL") {}
    // override
    // does not save toeknURI in the state var
    function _baseURI() internal pure override returns (string memory) {
        return "https://ipfs.io/ipfs/QmZSMw1PARg1dDxaN2CoNbskraRXJo7trUsoUickrRgrTn";
    }

    function mint(address to) public returns(uint256) {
        //require(_tokenIds.current() < 3); //only 3 to mint
        _tokenIds.increment(); // add 1 by minting
        uint256 newTokenId = _tokenIds.current();
        _mint(to, newTokenId); // or _safeMint(to, newTokenId);

        return newTokenId;
    }
}

Writing src/MyNFT.sol


In [8]:
!solc-windows.exe @openzeppelin/="C:\\Users\\youve\\Code\\201810820\\node_modules\\@openzeppelin\\" --optimize --combined-json abi,bin src/MyNFT.sol > src/MyNFT.json

In [11]:
%%writefile src/MyNFTUriOwner.sol
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC721/extensions/ERC721URIStorage.sol";
//import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/v4.4.0/contracts/token/ERC721/extensions/ERC721URIStorage.sol";
import "@openzeppelin/contracts/utils/Counters.sol";
//import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/v4.4.0/contracts/utils/Counters.sol";
import "@openzeppelin/contracts/access/Ownable.sol";
//import "https://github.com/nibbstack/erc721/blob/2.6.1/src/contracts/ownership/ownable.sol";

//Ownable added: minToPay, transferToPay
contract MyNFTUriOwner is ERC721URIStorage, Ownable {
    using Counters for Counters.Counter;
    Counters.Counter private _tokenIds;	//NFT id

    mapping(uint256 => uint256) public tranPrice;

    // from, to, tokenId, tokenURI, tranPrice
    event Transfer(address from, address to, uint256 _tokenId,
        string tokenURI, uint256 tranPrice);

    constructor() ERC721("MyNFT", "JSL") {}

    function mint(address to, string memory tokenURI) public returns(uint256) {
        _tokenIds.increment(); // add 1 by minting
        uint256 newTokenId = _tokenIds.current();
        _mint(to, newTokenId); // or _safeMint(to, newTokenId);
        _setTokenURI(newTokenId, tokenURI); // need to import ERC721URIStorage

        return newTokenId;
    }
    function mintToPay(address to, string memory tokenURI, uint256 _price)
    public payable onlyOwner returns(uint256) {
        uint256 newTokenId = mint(to, tokenURI);
        require(msg.value == _price, "Not Enough Ether");
        tranPrice[newTokenId] = _price;

        return newTokenId;
    }

    function getOwner() view public returns(address) {
        return owner();  //function of the Ownerable contract
    }

    // msg.value transferred to this contract!!
    function transferToPay(address from, address to, uint256 _tokenId) public payable {
        require(_exists(_tokenId), "Error: TokenId not owned");
        require(msg.value >= tranPrice[_tokenId], "Error: Token costs more");
        _transfer(from, to, _tokenId);
        emit Transfer(from, to, _tokenId, tokenURI(_tokenId), tranPrice[_tokenId]);
    }
}

Writing src/MyNFTUriOwner.sol


In [12]:
!solc-windows.exe @openzeppelin/="C:\\Users\\youve\\Code\\201810820\\node_modules\\@openzeppelin\\" --optimize --combined-json abi,bin src/MyNFTUriOwner.sol > src/MyNFTUriOwner.json

In [15]:
%%writefile src/MyNFTUriOwnerDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./MyNFTUriOwner.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts);
//console.log(contractName);
console.log("- contract name: ", contractName[12]);
_abiArray=_abiBinJson.contracts[contractName[12]].abi;
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin="0x"+_abiBinJson.contracts[contractName[12]].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1800000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/MyNFTUriOwnerDeploy.js


In [16]:
!node src/MyNFTUriOwnerDeploy.js

- contract name:  src/MyNFTUriOwner.sol:MyNFTUriOwner
Deploying the contract from 0x41fE5792b2FC66e84BE3e732d38b5dcDfD7834b5
hash: 0x1daf7957b1243564eaf929552f42fcc63958edcdbdf24a50301f8cba561ccbe7
---> The contract deployed to: 0x2f65d0f84c1A59b7812c41A14F7c478EFEfe7020


In [17]:
%%writefile src/MyNFTUriOwnerUse.js
var Web3=require('web3');
var _abiBinJson = require('./MyNFTUriOwner.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts);
console.log("- contract name: ", contractName[12]);
_abiArray=_abiBinJson.contracts[contractName[12]].abi;
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin="0x"+_abiBinJson.contracts[contractName[12]].bin;
var _nft = new web3.eth.Contract(_abiArray,"0x2f65d0f84c1A59b7812c41A14F7c478EFEfe7020");
var event = _nft.events.Transfer({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result.returnValues));
    }
});


async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("(0) Account: " + accounts[0]);
    var owner = await _nft.methods.getOwner().call();
    console.log("(1) owner: "+owner);
    console.log("(2) name: "+await _nft.methods.name().call());
    console.log("(3) symbol: "+await _nft.methods.symbol().call());
    var addrTo = accounts[1];
    var uri = "https://ipfs.io/ipfs/QmZSMw1PARg1dDxaN2CoNbskraRXJo7trUsoUickrRgrTn";
    //var uri = "https://ipfs.io/ipfs/QmZSMw1PARg1dDxaN2CoNbskraRXJo7trUsoUickrRgrTn";
    await _nft.methods.mint(addrTo,uri).send({from: accounts[0], gas: 1000000});
    var _tid = 1;
    console.log("(4) balanceOf "+owner+": "+await _nft.methods.balanceOf(owner).call());    
    console.log("(5) balanceOf "+addrTo+": "+await _nft.methods.balanceOf(addrTo).call());
    console.log("(6) ownerOf tokenId "+_tid+": "+await _nft.methods.ownerOf(_tid).call());
    //transfer from addrTo back to owner
    //console.log("(7) Who gets approved: "+await _nft.methods.getApproved(_tid).call());
    //await _nft.methods.approve(addrTo, _tid).send({from: owner, gas: 1000000});
    //console.log("(8) Who gets approved after calling approve: "+await _nft.methods.getApproved(_tid).call());
    await _nft.methods.transferFrom(addrTo, owner, _tid).send({from: addrTo, gas: 1000000});
    console.log("(9) balanceOf "+owner+": "+await _nft.methods.balanceOf(owner).call());    
    console.log("(10) balanceOf "+addrTo+": "+await _nft.methods.balanceOf(addrTo).call());
    console.log("(11) ownerOf after transferred: "+await _nft.methods.ownerOf(_tid).call());
    //process.exit(1); //force exit
}
doIt()

Writing src/MyNFTUriOwnerUse.js


In [18]:
!node src/MyNFTUriOwnerUse.js

- contract name:  src/MyNFTUriOwner.sol:MyNFTUriOwner
(0) Account: 0x41fE5792b2FC66e84BE3e732d38b5dcDfD7834b5
(1) owner: 0x41fE5792b2FC66e84BE3e732d38b5dcDfD7834b5
(2) name: MyNFT
(3) symbol: JSL
(4) balanceOf 0x41fE5792b2FC66e84BE3e732d38b5dcDfD7834b5: 0
(5) balanceOf 0xc6281deC16C579cD71a498cFE23007C4EeBa89A3: 1
(6) ownerOf tokenId 1: 0xc6281deC16C579cD71a498cFE23007C4EeBa89A3
(9) balanceOf 0x41fE5792b2FC66e84BE3e732d38b5dcDfD7834b5: 1
(10) balanceOf 0xc6281deC16C579cD71a498cFE23007C4EeBa89A3: 0
(11) ownerOf after transferred: 0x41fE5792b2FC66e84BE3e732d38b5dcDfD7834b5
